# **GANs for EEG signals**

**Implementing GANs for EEG signals involves several steps:**

* Preprocessing the EEG data: The first step is to preprocess the EEG data to remove noise and artifacts and extract relevant features. Common preprocessing techniques include filtering, artifact removal, and feature extraction.

* Building the generator and discriminator models: The generator and discriminator models are the core components of a GAN. The generator model takes a random noise vector as input and generates fake EEG samples, while the discriminator model takes real and fake EEG samples as input and classifies them as real or fake.

* Training the GAN: The GAN is trained using a training dataset of real EEG samples. The generator model is trained to generate EEG samples that can fool the discriminator model into classifying them as real, while the discriminator model is trained to distinguish between real and fake EEG samples. The two models are trained together in an adversarial manner, with the generator trying to generate more realistic samples and the discriminator trying to correctly classify the samples.

* Evaluating the GAN: Once the GAN is trained, it can be evaluated by generating new EEG samples and comparing them to real EEG samples. Evaluation metrics can include visual inspection of the generated samples, statistical analysis of the generated samples, and comparison of the generated samples to real EEG data in terms of their frequency content, temporal dynamics, and other features.

* Applying the GAN: The trained GAN can be used for various applications, such as generating synthetic EEG data for data augmentation, generating realistic EEG data for testing and validating EEG signal processing algorithms, and generating EEG data for simulating brain activity in computational models.

Implementing GANs for EEG signals can be a challenging task that requires expertise in signal processing, machine learning, and deep learning. It is important to carefully design and optimize the GAN architecture, preprocessing techniques, and training parameters to achieve high-quality results.

![Architecture-of-the-GAN-Noisy-EEG-to-clean-EEG](https://user-images.githubusercontent.com/68053788/224513303-544fc37a-c1a1-4c1c-843b-4bd67da376de.png)

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from generate_fake_eeg import generate_fake_eeg
from preprocess_eeg import preprocess_eeg
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, BatchNormalization, LeakyReLU, Dense, Input, Reshape


In [ ]:
def build_generator(input_shape):
    model_input = Input(shape=input_shape)
    x = Dense(64)(model_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((1, 64))(x)
    x = Conv1DTranspose(filters=64, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1DTranspose(filters=32, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1DTranspose(filters=1, kernel_size=4, strides=2, padding='same')(x)
    generator_output = Reshape((input_shape[0], input_shape[1]))(x)
    model = Model(model_input, generator_output, name='generator')
    return model

* The **build_generator** function is responsible for creating the generator component of the GANs model. The generator takes a random noise vector as an input and produces synthetic EEG data as output.

* The function takes an input_shape argument, which represents the shape of the input noise vector. The shape is a tuple of two integers: **(latent_dim, 1)**, where latent_dim is the dimensionality of the input noise vector.

* It then applies a fully connected layer with 64 units followed by a LeakyReLU activation function with a slope of 0.2. The output of this layer is reshaped to have a shape of (1, 64) using the Reshape layer.

* Next, the function applies a transposed convolutional layer with 64 filters, a kernel size of 4, a stride of 2, and padding set to 'same'. This is followed by a batch normalization layer and a LeakyReLU activation function with a slope of 0.2. The function repeats this process with another transposed convolutional layer with 32 filters and a kernel size of 4.

Finally, the function applies one more transposed convolutional layer with a single filter and a kernel size of 4. The output of this layer is then reshaped to have the same shape as the input EEG data, which is **(num_channels, num_samples)**.

In [ ]:
def build_discriminator(input_shape):
    model_input = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same')(model_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=128, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    discriminator_output = Dense(1, activation='sigmoid')(x)
    model = Model(model_input, discriminator_output, name='discriminator')
    return model

* The **build_discriminator** function is responsible for creating the discriminator component of the GANs model. The discriminator takes a set of EEG data as input and determines whether it is real (i.e. from the training data) or fake (i.e. generated by the generator).

* The function takes an input_shape argument, which represents the shape of the input EEG data. The shape is a tuple of two integers: **(num_channels, num_samples)**, where num_channels is the number of channels in the EEG data, and num_samples is the number of time samples.

* It then applies a convolutional layer with 32 filters, a kernel size of 4, a stride of 2, and padding set to 'same'. This is followed by a LeakyReLU activation function with a slope of 0.2. The output of this layer is then passed through another convolutional layer with 64 filters and a kernel size of 4, followed by a batch normalization layer and another LeakyReLU activation function.

* The function repeats this process with one more convolutional layer with 128 filters and a kernel size of 4, followed by another batch normalization layer and LeakyReLU activation function. The output of this layer is then flattened using the Flatten layer.

Finally, the function applies a fully connected layer with a single unit and a sigmoid activation function. The output of this layer represents the discriminator's decision on whether the input EEG data is real or fake.

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=generator.input_shape[1:])
    gan_output = discriminator(generator(gan_input))
    gan_model = Model(gan_input, gan_output, name='gan')
    gan_model.compile(loss='binary_crossentropy', optimizer='adam')
    return gan_model

* The **build_gan** function is responsible for building the entire GANs model, which consists of a generator and a discriminator.

* The first step in building the GANs model is to set the discriminator to be non-trainable. This is because we only want to train the generator in the first phase of training. During this phase, the discriminator is treated as a fixed, pre-trained model that is used to evaluate the generated EEG data.

* Next, the function creates an input layer. The shape of this layer is the same as the input shape of the generator model.

* The function then passes the input layer through the generator model to generate a set of fake EEG data. This generated data is then passed as input to the discriminator model to obtain the discriminator's decision on whether the generated EEG data is real or fake.

* The output of the discriminator is then used as the output of the GANs model. 

Finally, the function creates a Keras model object representing the GANs model, and compiles it using binary cross-entropy loss and the Adam optimizer.

In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tqdm import tqdm

# Define hyperparameters
input_shape = (32, 500) # shape of input signals
latent_dim = 100 # dimension of the latent space
batch_size = 128 # batch size for training
epochs = 100 # number of epochs to train for
sample_interval = 10 # interval for saving generated samples

# Load EEG data
eeg_data = np.load('eeg_data.npy') # shape (num_samples, num_channels, num_timesteps)

# Define models
generator = build_generator(input_shape=input_shape)
discriminator = build_discriminator(input_shape=input_shape)
gan = build_gan(generator, discriminator)

# Compile models
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

# Train models
for epoch in range(epochs):
    # ---------------------
    # Train discriminator
    # ---------------------
    # Select a random batch of real EEG signals
    idx = np.random.randint(0, eeg_data.shape[0], batch_size)
    real_signals = eeg_data[idx]

    # Generate a batch of fake EEG signals
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_signals = generator.predict(noise)

    # Label real and fake EEG signals
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    real_loss = discriminator.train_on_batch(real_signals, real_labels)
    fake_loss = discriminator.train_on_batch(fake_signals, fake_labels)
    d_loss = 0.5 * np.add(real_loss, fake_loss)

    # -----------------
    # Train generator
    # -----------------
    # Generate a batch of noise vectors
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Label fake EEG signals as real
    valid_labels = np.ones((batch_size, 1))

    # Train the generator
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress
    print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss:.4f} | G Loss: {g_loss:.4f}")

    # Save generated samples
    if epoch % sample_interval == 0:
        noise = np.random.normal(0, 1, (10, latent_dim))
        generated_signals = generator.predict(noise)
        np.save(f"generated_eeg_signals_epoch{epoch}.npy", generated_signals)


Then, the generator, discriminator, and GAN models are defined using the build_generator(), build_discriminator(), and build_gan() functions. The models are compiled using the Adam optimizer and binary cross-entropy loss function.

* The training loop begins with iterating through the specified number of epochs. Within each epoch, the discriminator is trained first by selecting a random batch of real EEG signals from the loaded data and generating a batch of fake EEG signals using the generator model. The real and fake signals are labeled accordingly and used to train the discriminator model using the train_on_batch() function. The loss of the discriminator is calculated as the average of the loss for the real and fake signals.

* Next, the generator is trained by generating a batch of noise vectors and labeling the fake EEG signals as real. The generator model is trained using the train_on_batch() function. The loss of the generator is then calculated.

* After each epoch, the current loss of the discriminator and generator are printed for progress monitoring. Additionally, if the current epoch is a multiple of the specified sample_interval, a random batch of noise vectors is generated and passed through the generator to produce fake EEG signals, which are then saved in a .npy file for later analysis.

This script loads the EEG data, defines the GAN models with the modified architecture for multiple channels, compiles the models, and trains the models for the specified number of epochs. The generated EEG signals are saved every sample_interval epochs. Note that you would need to modify this script to fit your specific use case, such as changing the input data shape and the output file names.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some fake EEG signals
num_samples = 100
latent_dim = 32
fake_latent = np.random.normal(0, 1, size=(num_samples, latent_dim))
fake_signals = generator.predict(fake_latent)

# Load some real EEG signals
real_signals = np.load('real_eeg_signals.npy')

# Calculate and print the mean squared error between real and fake signals
mse = np.mean(np.square(real_signals - fake_signals))
print(f'Mean Squared Error: {mse}')

# Plot a random sample of real and fake signals for visual comparison
num_plots = 5
indices = np.random.choice(num_samples, num_plots)
plt.figure(figsize=(10, 6))
for i, idx in enumerate(indices):
    plt.subplot(num_plots, 2, 2*i+1)
    plt.plot(real_signals[idx])
    plt.title('Real EEG Signal')
    plt.subplot(num_plots, 2, 2*i+2)
    plt.plot(fake_signals[idx])
    plt.title('Fake EEG Signal')
plt.tight_layout()
plt.show()

This script generates 100 fake signals using the generator model and loads some real signals from a file. It then calculates the mean squared error between the real and fake signals and prints it. Finally, it plots a random sample of real and fake signals side by side for visual comparison.